In [1]:
import numpy as np
import pandas as pd
import os
import sys


In [14]:
sys.path.insert(0, 'YOUR_PATH/fNIRS-mental-workload-classifiers/helpers')
from utils import load_pickle, save_pickle

In [3]:
def bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=1000, upper_percentile=97.5, lower_percentile=2.5):
    '''
    To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
    Then for each included subject, sample the chunk to calculate accuracy for this subject
    '''
    
    rng = np.random.RandomState(0)
    
    num_candidate_subjects = len(candidate_subjects)
    assert num_candidate_subjects == 68
    
#     return_dict = dict()
    accuracy_upper_percentile = [] # each element is the accuracy of this bootstrap sample (the average accuracy of the selected subjects with their selected chunks in this bootstrap sample)
    accuracy_lower_percentile = []
    accuracy_median = []
    
    #for each selected subject, independently resample the chunks to include (as the test set for this subject)
    for subject_id in candidate_subjects:
        print('subject_id: {}'.format(subject_id))
#         return_dict[subject_id] = dict()
        this_subject_accuracies = []
        #load the test predictions (for the selected hyper setting) of this subject, and the corresponding true labels
        ResultSaveDict_this_subject_path = lookup_table.loc[lookup_table['subject_id']==subject_id].experiment_folder.values[0]
        ResultSaveDict_this_subject = load_pickle(ResultSaveDict_this_subject_path, 'predictions/result_save_dict.pkl')

        TestLogits_this_subject = ResultSaveDict_this_subject['bestepoch_test_logits']
        TrueLabels_this_subject = ResultSaveDict_this_subject['bestepoch_test_class_labels']


        for i in range(num_bootstrap_samples):
            print('sample: {}'.format(i))

            #bootstrap the chunks to include for this subject (at this bootstrap sample)
            chunk_location_ix = np.array(range(len(TrueLabels_this_subject)))
            bootstrap_chunk_location_ix = rng.choice(chunk_location_ix, len(TrueLabels_this_subject), replace=True)
#             print('bootstrap_chunk_location_ix: {}'.format(bootstrap_chunk_location_ix))

            bootstrap_chunks_logits = TestLogits_this_subject[bootstrap_chunk_location_ix]
            bootstrap_chunks_true_labels = TrueLabels_this_subject[bootstrap_chunk_location_ix]

            accuracy_this_subject_this_bootstrap = (bootstrap_chunks_logits.argmax(1) == bootstrap_chunks_true_labels).mean()*100

            this_subject_accuracies.append(accuracy_this_subject_this_bootstrap)
        
        this_subject_upper_percentile = np.percentile(this_subject_accuracies, upper_percentile)
        this_subject_lower_percentile = np.percentile(this_subject_accuracies, lower_percentile)
        this_subject_median = np.percentile(this_subject_accuracies, 50)
        
        accuracy_upper_percentile.append(this_subject_upper_percentile)
        accuracy_lower_percentile.append(this_subject_lower_percentile)
        accuracy_median.append(this_subject_median)
        
    return accuracy_upper_percentile, accuracy_lower_percentile, accuracy_median




## DeepConvNet

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = 'YOUR_PATH/fNIRS-mental-workload-classifiers/experiments/generic_models/DeepConvNet/binary/'


In [5]:
OtherSubject64_summary_path = '64vs4_summary/'
OtherSubject16_summary_path = '16vs4_summary/'
OtherSubject4_summary_path = '4vs4_summary/'


In [6]:
OtherSubject64_df = pd.read_csv(os.path.join(root_path, OtherSubject64_summary_path, 'AllSubjects_summary.csv'))
OtherSubject16_df = pd.read_csv(os.path.join(root_path, OtherSubject16_summary_path, 'AllSubjects_summary.csv'))
OtherSubject4_df = pd.read_csv(os.path.join(root_path, OtherSubject4_summary_path, 'AllSubjects_summary.csv'))


In [7]:
OtherSubject64_df

,subject_id,bucket,max_validation_accuracy,corresponding_test_accuracy,performance_string,experiment_folder
0,56,TestBucket1,65.877,96.505376,highest validation accuracy: 65.87701612903226...,YOUR_PATH/NuripsDa...
1,72,TestBucket1,65.877,51.612903,highest validation accuracy: 65.87701612903226...,YOUR_PATH/NuripsDa...
2,86,TestBucket1,65.877,50.537634,highest validation accuracy: 65.87701612903226...,YOUR_PATH/NuripsDa...
3,79,TestBucket1,65.877,48.387097,highest validation accuracy: 65.87701612903226...,YOUR_PATH/NuripsDa...
4,82,TestBucket2,66.776,76.612903,highest validation accuracy: 66.77587365591397...,YOUR_PATH/NuripsDa...
...,...,...,...,...,...,...
63,35,TestBucket16,69.162,64.784946,highest validation accuracy: 69.16162634408603...,YOUR_PATH/NuripsDa...
64,21,TestBucket17,71.900,61.559140,highest validation accuracy: 71.90020161290323...,YOUR_PATH/NuripsDa...
65,60,TestBucket17,71.900,66.935484,highest validation accuracy: 71.90020161290323...,YOUR_PATH/NuripsDa...
66,29,TestBucket17,71.900,73.118280,highest validation accuracy: 71.90020161290323...,YOUR_PATH/NuripsDa...


### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=1000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 31, 32, 34,
 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49,  5, 51, 52,
 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 68, 69,  7, 70, 71,
 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 91, 92, 93,
 94, 95, 97])


In [9]:
len(candidate_subjects)

68

#### OtherSubject64

In [10]:
lookup_table = OtherSubject64_df.copy()
OtherSubject64_upper_percentile_list, OtherSubject64_lower_percentile_list, OtherSubject64_median_list = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)


subject_id: 1
sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample

sample: 92
sample: 93
sample: 94
sample: 95
sample: 96
sample: 97
sample: 98
sample: 99
sample: 100
sample: 101
sample: 102
sample: 103
sample: 104
sample: 105
sample: 106
sample: 107
sample: 108
sample: 109
sample: 110
sample: 111
sample: 112
sample: 113
sample: 114
sample: 115
sample: 116
sample: 117
sample: 118
sample: 119
sample: 120
sample: 121
sample: 122
sample: 123
sample: 124
sample: 125
sample: 126
sample: 127
sample: 128
sample: 129
sample: 130
sample: 131
sample: 132
sample: 133
sample: 134
sample: 135
sample: 136
sample: 137
sample: 138
sample: 139
sample: 140
sample: 141
sample: 142
sample: 143
sample: 144
sample: 145
sample: 146
sample: 147
sample: 148
sample: 149
sample: 150
sample: 151
sample: 152
sample: 153
sample: 154
sample: 155
sample: 156
sample: 157
sample: 158
sample: 159
sample: 160
sample: 161
sample: 162
sample: 163
sample: 164
sample: 165
sample: 166
sample: 167
sample: 168
sample: 169
sample: 170
sample: 171
sample: 172
sample: 173
sample: 174
sample: 175


sample: 198
sample: 199
sample: 200
sample: 201
sample: 202
sample: 203
sample: 204
sample: 205
sample: 206
sample: 207
sample: 208
sample: 209
sample: 210
sample: 211
sample: 212
sample: 213
sample: 214
sample: 215
sample: 216
sample: 217
sample: 218
sample: 219
sample: 220
sample: 221
sample: 222
sample: 223
sample: 224
sample: 225
sample: 226
sample: 227
sample: 228
sample: 229
sample: 230
sample: 231
sample: 232
sample: 233
sample: 234
sample: 235
sample: 236
sample: 237
sample: 238
sample: 239
sample: 240
sample: 241
sample: 242
sample: 243
sample: 244
sample: 245
sample: 246
sample: 247
sample: 248
sample: 249
sample: 250
sample: 251
sample: 252
sample: 253
sample: 254
sample: 255
sample: 256
sample: 257
sample: 258
sample: 259
sample: 260
sample: 261
sample: 262
sample: 263
sample: 264
sample: 265
sample: 266
sample: 267
sample: 268
sample: 269
sample: 270
sample: 271
sample: 272
sample: 273
sample: 274
sample: 275
sample: 276
sample: 277
sample: 278
sample: 279
sample: 280
samp

sample: 317
sample: 318
sample: 319
sample: 320
sample: 321
sample: 322
sample: 323
sample: 324
sample: 325
sample: 326
sample: 327
sample: 328
sample: 329
sample: 330
sample: 331
sample: 332
sample: 333
sample: 334
sample: 335
sample: 336
sample: 337
sample: 338
sample: 339
sample: 340
sample: 341
sample: 342
sample: 343
sample: 344
sample: 345
sample: 346
sample: 347
sample: 348
sample: 349
sample: 350
sample: 351
sample: 352
sample: 353
sample: 354
sample: 355
sample: 356
sample: 357
sample: 358
sample: 359
sample: 360
sample: 361
sample: 362
sample: 363
sample: 364
sample: 365
sample: 366
sample: 367
sample: 368
sample: 369
sample: 370
sample: 371
sample: 372
sample: 373
sample: 374
sample: 375
sample: 376
sample: 377
sample: 378
sample: 379
sample: 380
sample: 381
sample: 382
sample: 383
sample: 384
sample: 385
sample: 386
sample: 387
sample: 388
sample: 389
sample: 390
sample: 391
sample: 392
sample: 393
sample: 394
sample: 395
sample: 396
sample: 397
sample: 398
sample: 399
samp

sample: 440
sample: 441
sample: 442
sample: 443
sample: 444
sample: 445
sample: 446
sample: 447
sample: 448
sample: 449
sample: 450
sample: 451
sample: 452
sample: 453
sample: 454
sample: 455
sample: 456
sample: 457
sample: 458
sample: 459
sample: 460
sample: 461
sample: 462
sample: 463
sample: 464
sample: 465
sample: 466
sample: 467
sample: 468
sample: 469
sample: 470
sample: 471
sample: 472
sample: 473
sample: 474
sample: 475
sample: 476
sample: 477
sample: 478
sample: 479
sample: 480
sample: 481
sample: 482
sample: 483
sample: 484
sample: 485
sample: 486
sample: 487
sample: 488
sample: 489
sample: 490
sample: 491
sample: 492
sample: 493
sample: 494
sample: 495
sample: 496
sample: 497
sample: 498
sample: 499
sample: 500
sample: 501
sample: 502
sample: 503
sample: 504
sample: 505
sample: 506
sample: 507
sample: 508
sample: 509
sample: 510
sample: 511
sample: 512
sample: 513
sample: 514
sample: 515
sample: 516
sample: 517
sample: 518
sample: 519
sample: 520
sample: 521
sample: 522
samp

...

sample: 308
sample: 309
sample: 310
sample: 311
sample: 312
sample: 313
sample: 314
sample: 315
sample: 316
sample: 317
sample: 318
sample: 319
sample: 320
sample: 321
sample: 322
sample: 323
sample: 324
sample: 325
sample: 326
sample: 327
sample: 328
sample: 329
sample: 330
sample: 331
sample: 332
sample: 333
sample: 334
sample: 335
sample: 336
sample: 337
sample: 338
sample: 339
sample: 340
sample: 341
sample: 342
sample: 343
sample: 344
sample: 345
sample: 346
sample: 347
sample: 348
sample: 349
sample: 350
sample: 351
sample: 352
sample: 353
sample: 354
sample: 355
sample: 356
sample: 357
sample: 358
sample: 359
sample: 360
sample: 361
sample: 362
sample: 363
sample: 364
sample: 365
sample: 366
sample: 367
sample: 368
sample: 369
sample: 370
sample: 371
sample: 372
sample: 373
sample: 374
sample: 375
sample: 376
sample: 377
sample: 378
sample: 379
sample: 380
sample: 381
sample: 382
sample: 383
sample: 384
sample: 385
sample: 386
sample: 387
sample: 388
sample: 389
sample: 390
samp

sample: 480
sample: 481
sample: 482
sample: 483
sample: 484
sample: 485
sample: 486
sample: 487
sample: 488
sample: 489
sample: 490
sample: 491
sample: 492
sample: 493
sample: 494
sample: 495
sample: 496
sample: 497
sample: 498
sample: 499
sample: 500
sample: 501
sample: 502
sample: 503
sample: 504
sample: 505
sample: 506
sample: 507
sample: 508
sample: 509
sample: 510
sample: 511
sample: 512
sample: 513
sample: 514
sample: 515
sample: 516
sample: 517
sample: 518
sample: 519
sample: 520
sample: 521
sample: 522
sample: 523
sample: 524
sample: 525
sample: 526
sample: 527
sample: 528
sample: 529
sample: 530
sample: 531
sample: 532
sample: 533
sample: 534
sample: 535
sample: 536
sample: 537
sample: 538
sample: 539
sample: 540
sample: 541
sample: 542
sample: 543
sample: 544
sample: 545
sample: 546
sample: 547
sample: 548
sample: 549
sample: 550
sample: 551
sample: 552
sample: 553
sample: 554
sample: 555
sample: 556
sample: 557
sample: 558
sample: 559
sample: 560
sample: 561
sample: 562
samp

sample: 638
sample: 639
sample: 640
sample: 641
sample: 642
sample: 643
sample: 644
sample: 645
sample: 646
sample: 647
sample: 648
sample: 649
sample: 650
sample: 651
sample: 652
sample: 653
sample: 654
sample: 655
sample: 656
sample: 657
sample: 658
sample: 659
sample: 660
sample: 661
sample: 662
sample: 663
sample: 664
sample: 665
sample: 666
sample: 667
sample: 668
sample: 669
sample: 670
sample: 671
sample: 672
sample: 673
sample: 674
sample: 675
sample: 676
sample: 677
sample: 678
sample: 679
sample: 680
sample: 681
sample: 682
sample: 683
sample: 684
sample: 685
sample: 686
sample: 687
sample: 688
sample: 689
sample: 690
sample: 691
sample: 692
sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
samp

sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
sample: 778
sample: 779
sample: 780
sample: 781
sample: 782
sample: 783
sample: 784
sample: 785
sample: 786
sample: 787
sample: 788
sample: 789
sample: 790
sample: 791
sample: 792
sample: 793
sample: 794
sample: 795
sample: 796
sample: 797
sample: 798
sample: 799
sample: 800
sample: 801
sample: 802
sample: 803
sample: 804
sample: 805
sample: 806
sample: 807
sample: 808
sample: 809
sample: 810
sample: 811
sample: 812
sample: 813
sample: 814
sample: 815
sample: 816
sample: 817
sample: 818
sample: 819
sample: 820
sample: 821
sample: 822
sample: 823
sample: 824
sample: 825
sample: 826
sample: 827
sample: 828
sample: 829
sample: 830
sample: 831
sample: 832
sample: 833
sample: 834
sample: 835
samp

sample: 905
sample: 906
sample: 907
sample: 908
sample: 909
sample: 910
sample: 911
sample: 912
sample: 913
sample: 914
sample: 915
sample: 916
sample: 917
sample: 918
sample: 919
sample: 920
sample: 921
sample: 922
sample: 923
sample: 924
sample: 925
sample: 926
sample: 927
sample: 928
sample: 929
sample: 930
sample: 931
sample: 932
sample: 933
sample: 934
sample: 935
sample: 936
sample: 937
sample: 938
sample: 939
sample: 940
sample: 941
sample: 942
sample: 943
sample: 944
sample: 945
sample: 946
sample: 947
sample: 948
sample: 949
sample: 950
sample: 951
sample: 952
sample: 953
sample: 954
sample: 955
sample: 956
sample: 957
sample: 958
sample: 959
sample: 960
sample: 961
sample: 962
sample: 963
sample: 964
sample: 965
sample: 966
sample: 967
sample: 968
sample: 969
sample: 970
sample: 971
sample: 972
sample: 973
sample: 974
sample: 975
sample: 976
sample: 977
sample: 978
sample: 979
sample: 980
sample: 981
sample: 982
sample: 983
sample: 984
sample: 985
sample: 986
sample: 987
samp

In [15]:
save_pickle('Bootstrapping_for_GenericDCN_BandPassFilter_EachSubject/', 'OtherSubject64_upper_percentile.pkl', OtherSubject64_upper_percentile_list)
save_pickle('Bootstrapping_for_GenericDCN_BandPassFilter_EachSubject/', 'OtherSubject64_lower_percentile.pkl', OtherSubject64_lower_percentile_list)
save_pickle('Bootstrapping_for_GenericDCN_BandPassFilter_EachSubject/', 'OtherSubject64_median.pkl', OtherSubject64_median_list)
